In [44]:
import os
from bs4 import BeautifulSoup
import asyncio
from playwright.async_api import async_playwright
import time

In [30]:
%pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [31]:
%pip install playwright

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [32]:
!playwright install

In [45]:
SEASONS = list(range(2016,2023))

In [46]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [51]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries + 1):
        await asyncio.sleep(sleep * i)  # Use asyncio.sleep for async compatibility
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()  # Corrected variable name
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
                await browser.close()  # Ensure the browser is closed after operation
                break  # Exit the loop if successful
        except Exception as e:  # Consider broad exception handling or specific exceptions
            print(f"Attempt {i} failed with error: {str(e)}")
            continue
    return html
    

In [52]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    if html is None:
        print(f"Failed to retrieve HTML for season {season}")
        return  # Skip this season or handle the error as appropriate

    soup = BeautifulSoup(html, 'html.parser')  # Ensure you specify the parser
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links if 'href' in l.attrs]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        if html is None:
            print(f"Failed to retrieve HTML for URL: {url}")
            continue  # Skip this URL or handle the error as appropriate

        with open(save_path, "w+") as f:
            f.write(html)


In [53]:
for season in SEASONS:
    await scrape_season(season)

Attempt 1 failed with error: 
Attempt 2 failed with error: 
Attempt 3 failed with error: 
Failed to retrieve HTML for season 2016
Attempt 1 failed with error: 


CancelledError: 